# On Boarding

In [1]:
!pip install ipykernel ipywidgets

In [1]:
!curl -fsSL https://download.docker.com/linux/debian/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian bookworm stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

!sudo apt-get update
!sudo apt-get install docker-ce docker-ce-cli containerd.io

File '/usr/share/keyrings/docker-archive-keyring.gpg' exists. Overwrite? (y/N) 


gpg: signal Interrupt caught ... exiting
Get:1 file:/etc/apt/mirrors/debian.list Mirrorlist [30 B]
Get:2 file:/etc/apt/mirrors/debian-security.list Mirrorlist [39 B]             
Hit:7 https://packages.cloud.google.com/apt google-compute-engine-bookworm-stable InRelease
Hit:8 https://download.docker.com/linux/debian bookworm InRelease              
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bookworm InRelease     
Hit:3 https://deb.debian.org/debian bookworm InRelease
Hit:4 https://deb.debian.org/debian bookworm-updates InRelease
Hit:5 https://deb.debian.org/debian bookworm-backports InRelease
Hit:6 https://deb.debian.org/debian-security bookworm-security InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
docker-ce is already the newest version (5:27.4.0-1~debian.12~bookworm).
docker-ce-cli is already the newest version (5:27.4.0-1~debian.12~bookworm).
containerd.io is already the newest v

In [2]:
!sudo docker run hello-world
!sudo docker info | grep "Total Memory"


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

 Total Memory: 15.63GiB


In [ ]:
# /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)
# test -d ~/.linuxbrew && eval "$(~/.linuxbrew/bin/brew shellenv)"
# test -d /home/linuxbrew/.linuxbrew && eval "$(/home/linuxbrew/.linuxbrew/bin/brew shellenv"
# echo "eval \"\$($(brew --prefix)/bin/brew shellenv)\"" >> ~/.bashrc
# brew install hello
# brew install vespa-cli
# export VESPA_CLI_HOME=/tmp
# vespa config set target local

If previously ran the vespa docker, stop and remove it:

`sudo docker stop vespa`

`sudo docker rm vespa`


In [4]:
!sudo docker run --detach --name vespa --hostname vespa-container    --publish 8080:8080 --publish 19071:19071 vespaengine/vespa

a117203e8a7eeb3d5bf497f2a797fdebb861df869f4ddb76a038e0c633f0a572


In [6]:
# Verify that the Vespa deploy API is accessible:


!curl http://127.0.0.1:19071/status.html


<title>OK</title>


In [12]:
# To add the vespa to the jupyter
import os
os.environ["PATH"] += os.pathsep + "/home/linuxbrew/.linuxbrew/bin"


In [16]:
!vespa clone album-recommendation myapp && cd myapp


Using cached sample apps ...
Cloned into myapp


In [18]:
os.chdir("./myapp")
!vespa deploy --wait 300

Waiting up to 5m0s for deploy API...
Uploading application package... ⣽

Uploading application package... done;1m⣯

Success: Deployed '.' with session ID 2
Waiting up to 5m0s for deployment to converge...
Waiting up to 5m0s for cluster discovery...
Waiting up to 5m0s for container default...


In [19]:
!vespa feed ext/documents.jsonl

{
  "feeder.operation.count": 5,
  "feeder.seconds": 0.420,
  "feeder.ok.count": 5,
  "feeder.ok.rate": 5.000,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 5,
  "http.request.bytes": 724,
  "http.request.MBps": 0.001,
  "http.exception.count": 0,
  "http.response.count": 5,
  "http.response.bytes": 658,
  "http.response.MBps": 0.001,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 416,
  "http.response.latency.millis.avg": 416,
  "http.response.latency.millis.max": 416,
  "http.response.code.counts": {
    "200": 5
  }
}


In [20]:
!vespa query "select * from music where album contains 'head'" language=en-US

{
    "root": {
        "id": "toplevel",
        "relevance": 1.0,
        "fields": {
            "totalCount": 1
        },
        "coverage": {
            "coverage": 100,
            "documents": 5,
            "full": true,
            "nodes": 1,
            "results": 1,
            "resultsFull": 1
        },
        "children": [
            {
                "id": "id:mynamespace:music::a-head-full-of-dreams",
                "relevance": 0.16343879032006287,
                "source": "music",
                "fields": {
                    "sddocname": "music",
                    "documentid": "id:mynamespace:music::a-head-full-of-dreams",
                    "artist": "Coldplay",
                    "album": "A Head Full of Dreams",
                    "year": 2015,
                    "category_scores": {
                        "type": "tensor<float>(cat{})",
                        "cells": {
                            "pop": 1.0,
                            "rock":

In [21]:
!vespa query "select * from music where true" \
     "ranking=rank_albums" \
     "input.query(user_profile)={pop:0.8,rock:0.2,jazz:0.1}"

{
    "root": {
        "id": "toplevel",
        "relevance": 1.0,
        "fields": {
            "totalCount": 5
        },
        "coverage": {
            "coverage": 100,
            "documents": 5,
            "full": true,
            "nodes": 1,
            "results": 1,
            "resultsFull": 1
        },
        "children": [
            {
                "id": "id:mynamespace:music::a-head-full-of-dreams",
                "relevance": 0.2100000036880374,
                "source": "music",
                "fields": {
                    "sddocname": "music",
                    "documentid": "id:mynamespace:music::a-head-full-of-dreams",
                    "artist": "Coldplay",
                    "album": "A Head Full of Dreams",
                    "year": 2015,
                    "category_scores": {
                        "type": "tensor<float>(cat{})",
                        "cells": {
                            "pop": 1.0,
                            "rock": 

In [23]:
!vespa query "select * from music where true" \
     "ranking=rank_albums" \
     "input.query(user_profile)={pop:0.8,rock:0.2,jazz:0.1}" \
     "presentation.format.tensors=short-value"

{
    "root": {
        "id": "toplevel",
        "relevance": 1.0,
        "fields": {
            "totalCount": 5
        },
        "coverage": {
            "coverage": 100,
            "documents": 5,
            "full": true,
            "nodes": 1,
            "results": 1,
            "resultsFull": 1
        },
        "children": [
            {
                "id": "id:mynamespace:music::a-head-full-of-dreams",
                "relevance": 0.2100000036880374,
                "source": "music",
                "fields": {
                    "sddocname": "music",
                    "documentid": "id:mynamespace:music::a-head-full-of-dreams",
                    "artist": "Coldplay",
                    "album": "A Head Full of Dreams",
                    "year": 2015,
                    "category_scores": {
                        "pop": 1.0,
                        "rock": 0.20000000298023224,
                        "jazz": 0.0
                    }
                }
  

In [24]:
!vespa document get id:mynamespace:music::a-head-full-of-dreams

{
    "pathId": "/document/v1/mynamespace/music/docid/a-head-full-of-dreams",
    "id": "id:mynamespace:music::a-head-full-of-dreams",
    "fields": {
        "artist": "Coldplay",
        "year": 2015,
        "category_scores": {
            "type": "tensor<float>(cat{})",
            "cells": {
                "pop": 1.0,
                "rock": 0.20000000298023224,
                "jazz": 0.0
            }
        },
        "album": "A Head Full of Dreams"
    }
}


In [25]:
!vespa visit

{"id":"id:mynamespace:music::when-we-all-fall-asleep-where-do-we-go","fields":{"artist":"Billie Eilish","year":2019,"category_scores":{"type":"tensor<float>(cat{})","cells":{"pop":1.0,"rock":0.0,"jazz":0.10000000149011612}},"album":"When We All Fall Asleep, Where Do We Go?"}}
{"id":"id:mynamespace:music::liebe-ist-für-alle-da","fields":{"artist":"Rammstein","year":2009,"category_scores":{"type":"tensor<float>(cat{})","cells":{"pop":0.10000000149011612,"rock":1.0,"jazz":0.0}},"album":"Liebe ist für alle da"}}
{"id":"id:mynamespace:music::love-is-here-to-stay","fields":{"artist":"Diana Krall","year":2018,"category_scores":{"type":"tensor<float>(cat{})","cells":{"pop":0.4000000059604645,"rock":0.0,"jazz":0.800000011920929}},"album":"Love Is Here To Stay"}}
{"id":"id:mynamespace:music::a-head-full-of-dreams","fields":{"artist":"Coldplay","year":2015,"category_scores":{"type":"tensor<float>(cat{})","cells":{"pop":1.0,"rock":0.20000000298023224,"jazz":0.0}},"album":"A Head Full of Dreams"}}
